In [1]:
import pandas as pd
import numpy as np
# import torch
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
from sklearn import preprocessing, decomposition

seed = 42
FILENAME = "train_dataset.csv"

In [2]:
# 1. read_csv
# 2. dropna
# 3. drop label
# 4. replace src_bytes
# 5. casting 
# 6. encoding (cat/num)
# 7. extract y 
# 8. drop type
# 9. label encoding on y

import

In [3]:
# 1, 2 e 3
df = pd.read_csv(FILENAME, sep=",", low_memory=False)
df = df.dropna()
df = df.drop(columns=["label"])

# 4
df["src_bytes"] = df["src_bytes"].replace("0.0.0.0", np.nan).astype(float)
mean_src_bytes = df["src_bytes"].mean()
df["src_bytes"] = df["src_bytes"].fillna(mean_src_bytes)

# 5
df.astype({'src_bytes': 'int64', 'ts': 'datetime64[ms]', 'dns_AA': 'bool', 'dns_RD': 'bool', 'dns_RA': 'bool', 'dns_rejected': 'bool', 'ssl_resumed': 'bool', 'ssl_established': 'bool', 'weird_notice': 'bool'}).dtypes

# 6
y = df["type"]
df = df.drop(columns=["type"])

print(df.select_dtypes(include=['object']).shape)

# 7
oe = preprocessing.OrdinalEncoder()
df_oe = oe.fit_transform(df.select_dtypes(include=['object']))
df.loc[:, df.select_dtypes(include=['object']).columns] = df_oe
X = df.to_numpy()

# 8
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

(617002, 27)


splitting

In [4]:
indeces = np.arange(X.shape[0])
train_idx, val_idx = train_test_split(indeces, test_size=0.2, stratify=y, random_state=seed)
# val_idx, test_idx = train_test_split(val_idx, test_size=0.5, stratify=y[val_idx], random_state=seed)

fold = np.zeros(X.shape[0])
fold[train_idx] = -1
fold[val_idx] = 0
# fold[test_idx] = 1

ps = PredefinedSplit(fold)
ps.get_n_splits()

# X_test = X[test_idx,:]
# y_test = y[test_idx]
X_train = X[train_idx,:]
y_train = y[train_idx]
X_val = X[val_idx,:]
y_val = y[val_idx]

preprocess

In [5]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

X = scaler.transform(X)

pca = decomposition.PCA(n_components='mle', svd_solver='full')
pca.fit(X_train)

X = pca.transform(X)

In [6]:
print(pca.explained_variance_ratio_)

[1.44900607e-01 1.33969802e-01 1.33675083e-01 4.92563102e-02
 4.70841450e-02 4.61255229e-02 4.54983184e-02 4.23647696e-02
 3.90549396e-02 3.04763781e-02 2.33517088e-02 2.29889283e-02
 2.21947914e-02 2.17003153e-02 2.08727979e-02 2.00212553e-02
 1.97292747e-02 1.88016761e-02 1.76938953e-02 1.65682323e-02
 1.54616886e-02 1.51850213e-02 9.87827754e-03 7.51437108e-03
 6.96394655e-03 5.98062813e-03 5.02077679e-03 4.85400224e-03
 2.14769257e-03 2.05417698e-03 1.90607313e-03 1.43796611e-03
 1.20337627e-03 1.14801797e-03 9.00030610e-04 7.01092490e-04
 3.94912281e-04 3.89787902e-04 3.12938584e-04 1.14936749e-04
 1.01535087e-04]


In [7]:
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, balanced_accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pickle

In [8]:
# RandomForest
param_grid = {
    'n_estimators': [50, 100, 200],
    'criterion': ['gini', 'entropy']
}

scoring = ['balanced_accuracy', 'f1_weighted']

grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=ps, scoring=scoring, n_jobs=-1, verbose=10, refit='balanced_accuracy')
grid.fit(X, y)
print("---------------------------------")
print("Best hyper: ", grid.best_estimator_)
print("Best score: ", grid.best_score_)

Fitting 1 folds for each of 6 candidates, totalling 6 fits
[CV 1/1; 3/6] START criterion=gini, n_estimators=200............................
[CV 1/1; 1/6] START criterion=gini, n_estimators=50.............................
[CV 1/1; 5/6] START criterion=entropy, n_estimators=100.........................
[CV 1/1; 4/6] START criterion=entropy, n_estimators=50..........................
[CV 1/1; 2/6] START criterion=gini, n_estimators=100............................
[CV 1/1; 6/6] START criterion=entropy, n_estimators=200.........................
[CV 1/1; 1/6] END criterion=gini, n_estimators=50; balanced_accuracy: (test=0.998) f1_weighted: (test=0.999) total time= 3.5min
[CV 1/1; 4/6] END criterion=entropy, n_estimators=50; balanced_accuracy: (test=0.998) f1_weighted: (test=1.000) total time= 3.7min
[CV 1/1; 2/6] END criterion=gini, n_estimators=100; balanced_accuracy: (test=0.997) f1_weighted: (test=1.000) total time= 6.6min
[CV 1/1; 5/6] END criterion=entropy, n_estimators=100; balanced_acc

---

---

In [9]:
# 1, 2 e 3
df = pd.read_csv(FILENAME, sep=",", low_memory=False)
df = df.dropna()
df = df.drop(columns=["label"])

# 4
df["src_bytes"] = df["src_bytes"].replace("0.0.0.0", np.nan).astype(float)
mean_src_bytes = df["src_bytes"].mean()
df["src_bytes"] = df["src_bytes"].fillna(mean_src_bytes)

# 5
df.astype({'src_bytes': 'int64', 'ts': 'datetime64[ms]', 'dns_AA': 'bool', 'dns_RD': 'bool', 'dns_RA': 'bool', 'dns_rejected': 'bool', 'ssl_resumed': 'bool', 'ssl_established': 'bool', 'weird_notice': 'bool'}).dtypes

# 6
y = df["type"]
df = df.drop(columns=["type"])

print(df.select_dtypes(include=['object']).shape)

# 7
oe = preprocessing.OrdinalEncoder()
df_oe = oe.fit_transform(df.select_dtypes(include=['object']))
df.loc[:, df.select_dtypes(include=['object']).columns] = df_oe
X = df.to_numpy()

# 8
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

(617002, 27)


In [10]:
indeces = np.arange(X.shape[0])
train_idx, val_idx = train_test_split(indeces, test_size=0.2, stratify=y, random_state=seed)
# val_idx, test_idx = train_test_split(val_idx, test_size=0.5, stratify=y[val_idx], random_state=seed)

fold = np.zeros(X.shape[0])
fold[train_idx] = -1
fold[val_idx] = 0
# fold[test_idx] = 1

ps = PredefinedSplit(fold)
ps.get_n_splits()

# X_test = X[test_idx,:]
# y_test = y[test_idx]
X_train = X[train_idx,:]
y_train = y[train_idx]
X_val = X[val_idx,:]
y_val = y[val_idx]

In [11]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

X = scaler.transform(X)

# pca = decomposition.PCA(n_components='mle', svd_solver='full')
# pca.fit(X_train)

# X = pca.transform(X)

In [12]:
# RandomForest
param_grid = {
    'n_estimators': [50, 100, 200],
    'criterion': ['gini', 'entropy']
}

scoring = ['balanced_accuracy', 'f1_weighted']

grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=ps, scoring=scoring, n_jobs=-1, verbose=10, refit='balanced_accuracy')
grid.fit(X, y)
print("---------------------------------")
print("Best hyper: ", grid.best_estimator_)
print("Best score: ", grid.best_score_)

Fitting 1 folds for each of 6 candidates, totalling 6 fits
[CV 1/1; 5/6] START criterion=entropy, n_estimators=100.........................
[CV 1/1; 2/6] START criterion=gini, n_estimators=100............................
[CV 1/1; 3/6] START criterion=gini, n_estimators=200............................
[CV 1/1; 6/6] START criterion=entropy, n_estimators=200.........................
[CV 1/1; 1/6] START criterion=gini, n_estimators=50.............................
[CV 1/1; 4/6] START criterion=entropy, n_estimators=50..........................
[CV 1/1; 4/6] END criterion=entropy, n_estimators=50; balanced_accuracy: (test=0.996) f1_weighted: (test=1.000) total time=  21.8s
[CV 1/1; 1/6] END criterion=gini, n_estimators=50; balanced_accuracy: (test=0.996) f1_weighted: (test=1.000) total time=  22.0s
[CV 1/1; 2/6] END criterion=gini, n_estimators=100; balanced_accuracy: (test=0.997) f1_weighted: (test=1.000) total time=  38.4s
[CV 1/1; 5/6] END criterion=entropy, n_estimators=100; balanced_acc

---

---

In [60]:
# 1, 2 e 3
df = pd.read_csv(FILENAME, sep=",", low_memory=False)
df = df.dropna()
df = df.drop(columns=["label"])

# 4
df["src_bytes"] = df["src_bytes"].replace("0.0.0.0", np.nan).astype(float)
mean_src_bytes = df["src_bytes"].mean()
df["src_bytes"] = df["src_bytes"].fillna(mean_src_bytes)

# 5
df.astype({'src_bytes': 'int64', 'ts': 'datetime64[ms]', 'dns_AA': 'bool', 'dns_RD': 'bool', 'dns_RA': 'bool', 'dns_rejected': 'bool', 'ssl_resumed': 'bool', 'ssl_established': 'bool', 'weird_notice': 'bool'}).dtypes

# 6
y = df["type"]
df = df.drop(columns=["type"])

print(df.select_dtypes(include=['object']).shape)

# 7
oe = preprocessing.OrdinalEncoder()
df_oe = oe.fit_transform(df.select_dtypes(include=['object']))
df.loc[:, df.select_dtypes(include=['object']).columns] = df_oe
X = df.to_numpy()

# 8
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

In [ ]:
indeces = np.arange(X.shape[0])
train_idx, val_idx = train_test_split(indeces, test_size=0.2, stratify=y, random_state=seed)
# val_idx, test_idx = train_test_split(val_idx, test_size=0.5, stratify=y[val_idx], random_state=seed)

fold = np.zeros(X.shape[0])
fold[train_idx] = -1
fold[val_idx] = 0
# fold[test_idx] = 1

ps = PredefinedSplit(fold)
ps.get_n_splits()

# X_test = X[test_idx,:]
# y_test = y[test_idx]
X_train = X[train_idx,:]
y_train = y[train_idx]
X_val = X[val_idx,:]
y_val = y[val_idx]

In [14]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

X = scaler.transform(X)

# pca = decomposition.PCA(n_components='mle', svd_solver='full')
# pca.fit(X_train)

# X = pca.transform(X)

lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
lda.transform(X)

array([[-5.93179913e+00,  4.00022674e+00, -5.21043260e+00, ...,
        -1.35574797e-01,  3.79912781e+00, -3.88745483e+00],
       [-2.08636621e+00, -3.32270786e-01, -5.36121959e+00, ...,
        -6.07465354e+00, -1.99182280e+00, -3.75603546e-01],
       [-1.00060058e+00, -2.25348998e+00,  4.80439269e-03, ...,
         2.21397783e-01, -7.41315988e-01, -6.65059856e-01],
       ...,
       [ 3.02597595e+00, -2.43900247e+00, -7.13637327e-01, ...,
        -9.33429989e-01, -6.84175604e-01, -1.04748961e+00],
       [ 4.57737595e+00, -1.49195803e+00, -2.10543630e+00, ...,
        -1.85218219e+00,  4.36944697e+00,  1.91918828e+00],
       [ 3.44310869e+00, -2.21994280e+00, -8.77555638e-01, ...,
         1.14568784e+00, -1.37160912e-01, -9.56462870e-01]],
      shape=(617002, 9))

In [15]:
# RandomForest
param_grid = {
    'n_estimators': [50, 100, 200],
    'criterion': ['gini', 'entropy']
}

scoring = ['balanced_accuracy', 'f1_weighted']

grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=ps, scoring=scoring, n_jobs=-1, verbose=10, refit='balanced_accuracy')
grid.fit(X, y)
print("---------------------------------")
print("Best hyper: ", grid.best_estimator_)
print("Best score: ", grid.best_score_)

Fitting 1 folds for each of 6 candidates, totalling 6 fits
[CV 1/1; 2/6] START criterion=gini, n_estimators=100............................
[CV 1/1; 1/6] START criterion=gini, n_estimators=50.............................
[CV 1/1; 5/6] START criterion=entropy, n_estimators=100.........................
[CV 1/1; 4/6] START criterion=entropy, n_estimators=50..........................
[CV 1/1; 3/6] START criterion=gini, n_estimators=200............................
[CV 1/1; 6/6] START criterion=entropy, n_estimators=200.........................
[CV 1/1; 4/6] END criterion=entropy, n_estimators=50; balanced_accuracy: (test=0.997) f1_weighted: (test=1.000) total time=  22.6s
[CV 1/1; 1/6] END criterion=gini, n_estimators=50; balanced_accuracy: (test=0.998) f1_weighted: (test=1.000) total time=  23.0s
[CV 1/1; 5/6] END criterion=entropy, n_estimators=100; balanced_accuracy: (test=0.997) f1_weighted: (test=1.000) total time=  39.0s
[CV 1/1; 2/6] END criterion=gini, n_estimators=100; balanced_acc

---

---

---

In [16]:
# 1, 2 e 3
df = pd.read_csv(FILENAME, sep=",", low_memory=False)
df = df.dropna()
df = df.drop(columns=["label"])

# 4
df["src_bytes"] = df["src_bytes"].replace("0.0.0.0", np.nan).astype(float)
mean_src_bytes = df["src_bytes"].mean()
df["src_bytes"] = df["src_bytes"].fillna(mean_src_bytes)

# 5
df.astype({'src_bytes': 'int64', 'ts': 'datetime64[ms]', 'dns_AA': 'bool', 'dns_RD': 'bool', 'dns_RA': 'bool', 'dns_rejected': 'bool', 'ssl_resumed': 'bool', 'ssl_established': 'bool', 'weird_notice': 'bool'}).dtypes

# 6
y = df["type"]
df = df.drop(columns=["type"])

print(df.select_dtypes(include=['object']).shape)

# 7
oe = preprocessing.OrdinalEncoder()
df_oe = oe.fit_transform(df.select_dtypes(include=['object']))
df.loc[:, df.select_dtypes(include=['object']).columns] = df_oe
X = df.to_numpy()

# 8
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

(617002, 27)


In [17]:
indeces = np.arange(X.shape[0])
train_idx, val_idx = train_test_split(indeces, test_size=0.2, stratify=y, random_state=seed)
# val_idx, test_idx = train_test_split(val_idx, test_size=0.5, stratify=y[val_idx], random_state=seed)

fold = np.zeros(X.shape[0])
fold[train_idx] = -1
fold[val_idx] = 0
# fold[test_idx] = 1

ps = PredefinedSplit(fold)
ps.get_n_splits()

# X_test = X[test_idx,:]
# y_test = y[test_idx]
X_train = X[train_idx,:]
y_train = y[train_idx]
X_val = X[val_idx,:]
y_val = y[val_idx]

In [18]:
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

X = scaler.transform(X)

pca = decomposition.PCA(n_components='mle', svd_solver='full')
pca.fit(X_train)

X = pca.transform(X)

# lda = LinearDiscriminantAnalysis()
# lda.fit(X_train, y_train)
# lda.transform(X)

In [19]:
# SVC
param_grid = {
    'C': [0.1, 1, 10, 100, 1000], 
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto']  
}

scoring = ['balanced_accuracy', 'f1_weighted']

grid = GridSearchCV(SVC(), param_grid, cv=ps, scoring=scoring, n_jobs=-1, verbose=10, refit='balanced_accuracy')
grid.fit(X, y)
print("---------------------------------")
print("Best hyper: ", grid.best_estimator_)
print("Best score: ", grid.best_score_)

Fitting 1 folds for each of 40 candidates, totalling 40 fits
[CV 1/1; 8/40] START C=0.1, gamma=auto, kernel=sigmoid..........................
[CV 1/1; 3/40] START C=0.1, gamma=scale, kernel=poly............................
[CV 1/1; 7/40] START C=0.1, gamma=auto, kernel=poly.............................
[CV 1/1; 4/40] START C=0.1, gamma=scale, kernel=sigmoid.........................[CV 1/1; 6/40] START C=0.1, gamma=auto, kernel=rbf..............................

[CV 1/1; 5/40] START C=0.1, gamma=auto, kernel=linear...........................
[CV 1/1; 1/40] START C=0.1, gamma=scale, kernel=linear..........................
[CV 1/1; 2/40] START C=0.1, gamma=scale, kernel=rbf.............................
[CV 1/1; 1/40] END C=0.1, gamma=scale, kernel=linear; balanced_accuracy: (test=0.942) f1_weighted: (test=0.988) total time=28.4min
[CV 1/1; 9/40] START C=1, gamma=scale, kernel=linear............................
[CV 1/1; 5/40] END C=0.1, gamma=auto, kernel=linear; balanced_accuracy: (test=0

KeyboardInterrupt: 

---

In [ ]:
# 1, 2 e 3
df = pd.read_csv(FILENAME, sep=",", low_memory=False)
df = df.dropna()
df = df.drop(columns=["label"])

# 4
df["src_bytes"] = df["src_bytes"].replace("0.0.0.0", np.nan).astype(float)
mean_src_bytes = df["src_bytes"].mean()
df["src_bytes"] = df["src_bytes"].fillna(mean_src_bytes)

# 5
df.astype({'src_bytes': 'int64', 'ts': 'datetime64[ms]', 'dns_AA': 'bool', 'dns_RD': 'bool', 'dns_RA': 'bool', 'dns_rejected': 'bool', 'ssl_resumed': 'bool', 'ssl_established': 'bool', 'weird_notice': 'bool'}).dtypes

# 6
y = df["type"]
df = df.drop(columns=["type"])

print(df.select_dtypes(include=['object']).shape)

# 7
oe = preprocessing.OrdinalEncoder()
df_oe = oe.fit_transform(df.select_dtypes(include=['object']))
df.loc[:, df.select_dtypes(include=['object']).columns] = df_oe
X = df.to_numpy()

# 8
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)


indeces = np.arange(X.shape[0])
train_idx, val_idx = train_test_split(indeces, test_size=0.2, stratify=y, random_state=seed)
# val_idx, test_idx = train_test_split(val_idx, test_size=0.5, stratify=y[val_idx], random_state=seed)

fold = np.zeros(X.shape[0])
fold[train_idx] = -1
fold[val_idx] = 0
# fold[test_idx] = 1

ps = PredefinedSplit(fold)
ps.get_n_splits()

# X_test = X[test_idx,:]
# y_test = y[test_idx]
X_train = X[train_idx,:]
y_train = y[train_idx]
X_val = X[val_idx,:]
y_val = y[val_idx]


# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

X = scaler.transform(X)

# pca = decomposition.PCA(n_components='mle', svd_solver='full')
# pca.fit(X_train)

# X = pca.transform(X)

# lda = LinearDiscriminantAnalysis()
# lda.fit(X_train, y_train)
# lda.transform(X)


# SVC
param_grid = {
    'C': [0.1, 1, 10, 100, 1000], 
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto']  
}

scoring = ['balanced_accuracy', 'f1_weighted']

grid = GridSearchCV(SVC(), param_grid, cv=ps, scoring=scoring, n_jobs=-1, verbose=10, refit='balanced_accuracy')
grid.fit(X, y)
print("---------------------------------")
print("Best hyper: ", grid.best_estimator_)
print("Best score: ", grid.best_score_)

---

In [ ]:
# 1, 2 e 3
df = pd.read_csv(FILENAME, sep=",", low_memory=False)
df = df.dropna()
df = df.drop(columns=["label"])

# 4
df["src_bytes"] = df["src_bytes"].replace("0.0.0.0", np.nan).astype(float)
mean_src_bytes = df["src_bytes"].mean()
df["src_bytes"] = df["src_bytes"].fillna(mean_src_bytes)

# 5
df.astype({'src_bytes': 'int64', 'ts': 'datetime64[ms]', 'dns_AA': 'bool', 'dns_RD': 'bool', 'dns_RA': 'bool', 'dns_rejected': 'bool', 'ssl_resumed': 'bool', 'ssl_established': 'bool', 'weird_notice': 'bool'}).dtypes

# 6
y = df["type"]
df = df.drop(columns=["type"])

print(df.select_dtypes(include=['object']).shape)

# 7
oe = preprocessing.OrdinalEncoder()
df_oe = oe.fit_transform(df.select_dtypes(include=['object']))
df.loc[:, df.select_dtypes(include=['object']).columns] = df_oe
X = df.to_numpy()

# 8
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)


indeces = np.arange(X.shape[0])
train_idx, val_idx = train_test_split(indeces, test_size=0.2, stratify=y, random_state=seed)
# val_idx, test_idx = train_test_split(val_idx, test_size=0.5, stratify=y[val_idx], random_state=seed)

fold = np.zeros(X.shape[0])
fold[train_idx] = -1
fold[val_idx] = 0
# fold[test_idx] = 1

ps = PredefinedSplit(fold)
ps.get_n_splits()

# X_test = X[test_idx,:]
# y_test = y[test_idx]
X_train = X[train_idx,:]
y_train = y[train_idx]
X_val = X[val_idx,:]
y_val = y[val_idx]


from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

X = scaler.transform(X)

# pca = decomposition.PCA(n_components='mle', svd_solver='full')
# pca.fit(X_train)

# X = pca.transform(X)

lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
lda.transform(X)


# SVC
param_grid = {
    'C': [0.1, 1, 10, 100, 1000], 
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
    'gamma': ['scale', 'auto']  
}

scoring = ['balanced_accuracy', 'f1_weighted']

grid = GridSearchCV(SVC(), param_grid, cv=ps, scoring=scoring, n_jobs=-1, verbose=10, refit='balanced_accuracy')
grid.fit(X, y)
print("---------------------------------")
print("Best hyper: ", grid.best_estimator_)
print("Best score: ", grid.best_score_)

---
svc

In [62]:
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1, 10, 100, 1000],  
}  

grid = GridSearchCV(svm.LinearSVC(random_state=seed), param_grid, verbose=10, cv=ps, scoring='balanced_accuracy', n_jobs=-1)
grid.fit(X, y)
file = open("svm.save", "wb")
pickle.dump(grid.best_estimator_, file)
print("Best hyper: ", grid.best_estimator_)
print("Best score: ", grid.best_score_)
file.close()

Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV 2/2; 4/10] START C=1, penalty=l2............................................
[CV 1/2; 3/10] START C=1, penalty=l1............................................
[CV 1/2; 1/10] START C=0.1, penalty=l1..........................................
[CV 2/2; 3/10] START C=1, penalty=l1............................................[CV 2/2; 2/10] START C=0.1, penalty=l2..........................................

[CV 2/2; 1/10] START C=0.1, penalty=l1..........................................[CV 1/2; 4/10] START C=1, penalty=l2............................................

[CV 1/2; 2/10] START C=0.1, penalty=l2..........................................


/Users/filippobrajucha/Developer/data-analytics/.env/lib/python3.12/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 2/2; 1/10] END ...........C=0.1, penalty=l1;, score=0.649 total time=59.8min
[CV 1/2; 5/10] START C=10, penalty=l1...........................................


/Users/filippobrajucha/Developer/data-analytics/.env/lib/python3.12/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 1/2; 1/10] END ...........C=0.1, penalty=l1;, score=0.650 total time=62.7min
[CV 2/2; 5/10] START C=10, penalty=l1...........................................


...

/Users/filippobrajucha/Developer/data-analytics/.env/lib/python3.12/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 1/2; 10/10] END .........C=1000, penalty=l2;, score=0.723 total time=80.2min


/Users/filippobrajucha/Developer/data-analytics/.env/lib/python3.12/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[CV 2/2; 10/10] END .........C=1000, penalty=l2;, score=0.719 total time=72.9min
Best hyper:  LinearSVC(C=1000, random_state=42)
Best score:  0.7207091302697002


/Users/filippobrajucha/Developer/data-analytics/.env/lib/python3.12/site-packages/sklearn/svm/_base.py:1243: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [63]:
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [10000, 100000, 1000000],  
}  

grid = GridSearchCV(svm.LinearSVC(random_state=seed), param_grid, verbose=10, cv=ps, scoring='balanced_accuracy', n_jobs=-1)
grid.fit(X, y)
file = open("svm_.save", "wb")
pickle.dump(grid.best_estimator_, file)
print("Best hyper: ", grid.best_estimator_)
print("Best score: ", grid.best_score_)
file.close()

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV 1/2; 2/6] START C=10000, penalty=l2.........................................[CV 2/2; 2/6] START C=10000, penalty=l2.........................................
[CV 1/2; 4/6] START C=100000, penalty=l2........................................
[CV 2/2; 1/6] START C=10000, penalty=l1.........................................
[CV 1/2; 3/6] START C=100000, penalty=l1........................................
[CV 1/2; 1/6] START C=10000, penalty=l1.........................................
[CV 2/2; 4/6] START C=100000, penalty=l2........................................
[CV 2/2; 3/6] START C=100000, penalty=l1........................................



KeyboardInterrupt: 